This is the 3rd Lab for the course of Data Science Labs and Methods

In [15]:
import pandas
import mlxtend 

In [16]:
import csv
headers = []
rows = []
#skipped_headers = False
with open('retail_new.csv') as f:
    reading = csv.reader(f)
    headers = next(reading)
    for row in reading:
        if len(row) == 8 and row[0][0] != 'C':
            #we read the row and store it into the list
            rows.append(row)

In [17]:
#just checking to see if the data was read correctly 
print(rows[0])
print(headers)

['536365', '85123A', 'WHITE HANGING HEART T-LIGHT HOLDER', '6', '12/1/2010 8:26', '2.55', '17850', 'United Kingdom']
['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate', 'UnitPrice', 'CustomerID', 'Country']


In [18]:
#now we want to aggregate the items at an invoice level, so we have all of the items/products
#purchased from one invoice, together as a list or a set for more convenience. (depending on dataset, 
# it may be better to go with a list in some cases) 
from collections import defaultdict
invoices = defaultdict(lambda: set())
for row in rows:
    invoices[row[0]].add(row[2])

#the one below is the same but in the form of a list.
invoices1 =  defaultdict(lambda: list())
for row in rows:
    invoices1[row[0]].append(row[2])


Now, we build a matrix that conforms with the MLXtend library. 
For each column==Product, we check if the invoice==row, has that in its list. 
IF IT DOES, WE PUT A VALUE OF 1 for the i, j (row, column) configuration. 

In [19]:
#first, we get all of the products and put them in a set to avoid repetition
all_products = set()
#note, we use update so we can add multiple products from each of the lists directly
for p in invoices.values():
    all_products.update(p)
#Now, we have all the products as well. 
all_products = list(all_products) 
matrix = []
for p_list in invoices.values():
    m_row = [int(item in p_list) for item in all_products]
    matrix.append(m_row)
len(invoices)
#def get_presence_matrix(invoices, all_items):
 #   item_pos_dict = { k: v for v, k in enumerate(all_items) }
  #  presence_matrix = []
   # for invoice in invoices.values():
    #    row = [0] * len(all_items)
     #   for item in invoice:
      #      row[item_pos_dict[item]] = 1
       # presence_matrix.append(row)
    #return presence_matrix
#presence_matrix = get_presence_matrix(invoices, all_items)

22064

In [20]:
import pandas as pd
df = pd.DataFrame(data=matrix, columns=all_products)

In [21]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
from mlxtend.frequent_patterns import fpgrowth
for minsup in [0.5, 0.1, 0.05, 0.02, 0.01]:
    fi = fpgrowth(df, minsup)
    print(f"For {minsup} -> {len(fi)}")
    

For 0.5 -> 0
For 0.1 -> 1
For 0.05 -> 23
For 0.02 -> 303
For 0.01 -> 1472


In [24]:
freq_itemsets = fpgrowth(df, 0.02)
freq_itemsets = freq_itemsets[freq_itemsets["itemsets"].map(len) > 1]
#If we observe the result that this gives us, we can note which items are frequently bought together
#or bought as a bundle by customers